In [109]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [110]:
def load_xls_report(filename):
    data = pd.read_excel(filename, header=None)
    start_index = data[data.iloc[:, 0]=='Transacciones'].index[0]
    end_index = data.iloc[start_index:, 0].isna().idxmax()
    columns = data.iloc[start_index+1].tolist()
    data.columns = columns
    balance = data['Beneficio'].iloc[start_index+2]
    data = data.iloc[start_index+3:end_index-1,:]
    resultados = []
    for i in range(0, len(data) - 1, 2):
        fila_in = data.iloc[i]
        fila_out = data.iloc[i + 1]
        if fila_in['Dirección'] == 'in' and fila_out['Dirección'] == 'out':
            precio_entrada = fila_in['Precio']
            precio_salida = fila_out['Precio']
            beneficio = fila_out['Beneficio'] - abs(fila_in['Comisión']) - abs(fila_out['Comisión'])
            balance += beneficio
            resultados.append({
                'Fecha': fila_in['Fecha/Hora'],
                'Símbolo': fila_in['Símbolo'],
                'Precio Entrada': precio_entrada,
                'Precio Salida': precio_salida,
                'Volumen': fila_in['Volumen '],
                'Beneficio': beneficio,
                'Balance': balance
            })
    return pd.DataFrame(resultados)

In [111]:
filename='ReportTester-4000009439.xlsx'
df = load_xls_report(filename)
df.tail()

,Fecha,Símbolo,Precio Entrada,Precio Salida,Volumen,Beneficio,Balance
2816,2024.10.01 08:40:09,GDAXI,19392.2,19404.9,0.01,1.21,3727.78
2817,2024.10.01 10:11:33,GDAXI,19379.9,19391.2,0.01,1.07,3728.85
2818,2024.10.02 04:30:30,GDAXI,19246.7,19239.0,0.01,0.71,3729.56
2819,2024.10.02 06:33:36,GDAXI,19265.7,19253.0,0.01,1.21,3730.77
2820,2024.10.02 10:42:17,GDAXI,19251.0,19217.7,0.01,3.27,3734.04


In [112]:
total_trades = df.shape[0]
profit_trades = df[df['Beneficio']>0].shape[0]
profit_avg = df[df['Beneficio']>0]['Beneficio'].mean()
loss_trades = df[df['Beneficio']<0].shape[0]
loss_avg = df[df['Beneficio']<0]['Beneficio'].mean()
beneficio_neto  = (profit_avg*profit_trades)+(loss_avg*loss_trades)
total_trades = profit_trades + loss_trades
expected_profit = beneficio_neto / total_trades
profit_avg_points = abs(df[df['Beneficio']>0]['Precio Entrada'] - df[df['Beneficio']>0]['Precio Salida']).mean()
loss_avg_points = abs(df[df['Beneficio']<0]['Precio Entrada'] - df[df['Beneficio']<0]['Precio Salida']).mean()
punto_equilibrio = abs(loss_avg)/(abs(loss_avg)+profit_avg)
win_rates = profit_trades/total_trades

In [113]:
print(f"Número de operaciones totales: {total_trades}")
print(f"Número de operaciones ganadoras: {profit_trades}")
print(f"Número de operaciones perdedoras: {loss_trades}")
print(f"Media de puntos en operaciones ganadoras {profit_avg_points:.2f} puntos")
print(f"Media de puntos en operaciones perdedoras {loss_avg_points:.2f} puntos")
print(f"Ganancia media: {profit_avg:.2f}€")
print(f"Pérdida media: {loss_avg:.2f}€")
print(f"Rentabilidad esperada: {expected_profit:.4f}€")
print(f"Porcentaje de aciertos actual: {win_rates*100.0:.2f}%")
print(f"Porcentaje de aciertos para equilibrio: {punto_equilibrio*100.0:.2f}%")

Número de operaciones totales: 2819
Número de operaciones ganadoras: 1713
Número de operaciones perdedoras: 1106
Media de puntos en operaciones ganadoras 22.06 puntos
Media de puntos en operaciones perdedoras 35.06 puntos
Ganancia media: 2.15€
Pérdida media: -3.56€
Rentabilidad esperada: -0.0943€
Porcentaje de aciertos actual: 60.77%
Porcentaje de aciertos para equilibrio: 62.42%


In [114]:
df.tail()

,Fecha,Símbolo,Precio Entrada,Precio Salida,Volumen,Beneficio,Balance
2816,2024.10.01 08:40:09,GDAXI,19392.2,19404.9,0.01,1.21,3727.78
2817,2024.10.01 10:11:33,GDAXI,19379.9,19391.2,0.01,1.07,3728.85
2818,2024.10.02 04:30:30,GDAXI,19246.7,19239.0,0.01,0.71,3729.56
2819,2024.10.02 06:33:36,GDAXI,19265.7,19253.0,0.01,1.21,3730.77
2820,2024.10.02 10:42:17,GDAXI,19251.0,19217.7,0.01,3.27,3734.04
